In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType, sum, max, col, concat, lit, monotonically_increasing_id
import sys
import os



from pyspark import Row


from datetime import datetime,timedelta

from fbprophet import Prophet
import pandas as pd
import numpy as np

In [ ]:
def dfZipWithIndex (df, offset=1, colName="rowId"):
    '''
        Enumerates dataframe rows is native order, like rdd.ZipWithIndex(), but on a dataframe 
        and preserves a schema

        :param df: source dataframe
        :param offset: adjustment to zipWithIndex()'s index
        :param colName: name of the index column
    '''

    new_schema = StructType(
                    [StructField(colName,LongType(),True)]        # new added field in front
                    + df.schema.fields                            # previous schema
                )

    zipped_rdd = df.rdd.zipWithIndex()

    new_rdd = zipped_rdd.map(lambda args: ([args[1] + offset] + list(args[0])))

    return spark.createDataFrame(new_rdd, new_schema)

In [ ]:
schema = StructType([
        StructField("ds", DateType(), True),
        StructField("yhat", DoubleType(), True)
    ])

In [ ]:
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def fit_pandas_udf(df):
    """
    :param df: Dataframe (train + test data)
    :return: predictions as defined in the output schema
    """

    def train_fitted_prophet(df, cutoff):
        
        names = df.columns
        
        #train
        ts_train = (df
                    .query('id <= @cutoff')
                    .rename(columns={names[1]: 'ds', names[2]: 'y'})
                    .sort_values('ds')
                    )[['ds','y']]
        
        print(ts_train.columns)
        
        
        # test
        ts_test = (df
                   .query('id > @cutoff')
                   .rename(columns={names[1]: 'ds', names[2]: 'y'})
                   .sort_values('ds')
                   .assign(ds=lambda x: pd.to_datetime(x["ds"]))
                   .drop('y', axis=1)
                   )[['ds']]
        
        print(ts_test.columns)

 

        # init model
        m = Prophet(yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True)
        m.fit(ts_train)
        
        

        # to date
        
        # at this step we predict the future and we get plenty of additional columns be cautious
        ts_hat = (m.predict(ts_test)[["ds", "yhat"]]
                  .assign(ds=lambda x: pd.to_datetime(x["ds"]))
                  ).merge(ts_test, on=["ds"], how="left")  
        

        return pd.DataFrame(ts_hat, columns=schema.fieldNames())

    return train_fitted_prophet(df, cutoff)

In [ ]:
if __name__ == '__main__':
    spark = (SparkSession
             .builder
             .appName("forecasting")
             .getOrCreate()
             #.config('spark.sql.execution.arrow.enable', 'true')
             )
    
    data = (spark
                .read
                .format("csv")
                .option('header', 'true')
                .option('inferSchema','true')
                .load('Downloads/AEP_hourly.csv')
                #.load('data_simulation.csv')
                
            )
    
    data.createOrReplaceTempView("data")
    data = spark.sql(f"SELECT LEFT(Datetime,10) AS Datetime, {data.columns[1]}  FROM data")
    data = data.groupBy("Datetime")\
               .mean("AEP_MW")\
               .sort(col('DateTime'))
    
    
    # 70% of the real dataset
    data_length = data.count()
    train_size = int(round(0.7 * data_length,0))
    
    
    ##Add future days to predict
    
    #last_day = data.tail(1)[0].__getitem__("Datetime")  # Não sei se é viável
    last_day = data.tail(1)[0].asDict()['Datetime']
    future_days = pd.date_range(start = last_day,
                                periods = 28)
    sequence_days = list(future_days.strftime("%Y-%m-%d"))[2:-1]
    future = spark.createDataFrame(sequence_days, 
                                   StringType())
    future.createOrReplaceTempView("future")
    future = spark.sql("SELECT value AS Datetime FROM future")
    future = future.withColumn(data.columns[1],
                               lit(None))
    

    
    df = (data.union(future)).sort(col('Datetime'))
    df = dfZipWithIndex(df,colName="id")
    
    
    
    cutoff = train_size
    # Apply forcasting
    global_predictions = (df
                          .groupBy()
                          .apply(fit_pandas_udf)
                          )

In [ ]:
global_predictions.show(2000)

In [ ]:
    data = (spark
                .read
                .format("csv")
                .option('header', 'true')
                .option('inferSchema','true')
                .load('data_simulation.csv')
                #.load('Downloads/AEP_hourly.csv')
            )
    data.count()

In [ ]:
allfiles =  spark\
            .read\
            .option("header","false")\
            .option('inferSchema','true')\
            .csv(f"{os.getcwd()}/abc/part*.csv")\
            .selectExpr("_c0 as Datetime","_c1 as MW")\
            .sort(col("Datetime"))

In [ ]:
allfiles.groupBy("Datetime").mean("MW").show(100)
